In [1]:
import csv
import urllib
import pandas as pd
import numpy as np
import sys
import datetime
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data as pdr
from googlefinance.client import get_price_data, get_prices_data, get_prices_time_data
import fix_yahoo_finance as yf
import pyodbc 
import os

In [2]:
yf.pdr_override()

In [7]:
#Download lists of all companies in All US exchanges.  Catch Error if urls become outdated.  
NASDAQ_url = 'http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download'
NYSE_url = 'http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NYSE&render=download'
AMEX_url = 'http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=AMEX&render=download'
try:
    response = urllib.request.urlopen(NASDAQ_url)
    NASDAQ_df = pd.read_csv(response)
    
    response = urllib.request.urlopen(NYSE_url)
    NYSE_df = pd.read_csv(response)
    
    response = urllib.request.urlopen(AMEX_url)
    AMEX_df = pd.read_csv(response)
    
    df=NASDAQ_df
    df=df.append(NYSE_df)
    df=df.append(AMEX_df)

    df=df.drop_duplicates('Symbol')
    df=df.set_index('Symbol')
    df=df.sort_values(by=['MarketCap'],ascending=False)
    
    #Only want companies with positive market cap
    df=df[df['MarketCap']>0]
    
except Exception:
    print("Download Fail")

In [4]:
end_date=today=datetime.datetime.now()
start_date= '2000-01-01'
directory='saved_data'
#result=fill_history(start_date,end_date,df,directory,10)

In [45]:
directory = r'C:\Users\Chris Atkeson\Documents\Market_Data\saved_data'

cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=localhost\SQLEXPRESS;"
                      "Database=Stock_Data;"
                      "Trusted_Connection=yes;")

#write_to_sql(cnxn,directory)

In [8]:
def fill_history(start_date,end_date,df,directory,num_runs):
    
    failed_downloads=[]
    success_downloads=[]

    for i,row in df.iterrows():
        symbol=i
        market_cap=row['MarketCap']
        nasdaq_price=row['LastSale']
        sector=row['Sector']
    
        if market_cap > 0:
            try:
                df_yahoo=pdr.get_data_yahoo(symbol,start_date,end_date)
                df_yahoo['ticker']=symbol
                df_yahoo=df_yahoo.set_index('Date')
                df_yahoo.to_csv(directory + '\\' + symbol +'.csv')
                success_downloads.append(symbol)
            except Exception as e:
                failed_downloads.append(symbol)
    
    for x in range (0,num_runs):
        
        failed_downloads=fill_hist_help(start_date,end_date,failed_downloads,directory)
        
    return (failed_downloads,success_downloads)
            

In [9]:
def fill_hist_help(start_date,end_date,symbols,directory):
    failed_downloads=[]
    for symbol in symbols:
        try:
            df_yahoo=pdr.get_data_yahoo(symbol,start_date,end_date)
            df_yahoo['ticker']=symbol
            df_yahoo=df_yahoo.set_index('Date')
            df_yahoo.to_csv(directory + '\\' + symbol +'.csv')
        except Exception as e:
            failed_downloads.append(symbol)
    return failed_downloads
    

In [43]:
def write_to_sql(cnxn,directory):
    #count=1
    cursor = cnxn.cursor()
    SQL_string=r"""Truncate table [stock_Data].[dbo].[Hist_Data]"""
    cursor.execute(SQL_string)
    cursor.close()
    
    for filename in os.listdir(directory):
        
        #just for general progress tricking
        #count+=1
        #if count % 100 == 0:
            #print(count)

        if filename.endswith(".csv"): 
            try:
                ticker=filename[:-4]
                csv_path= "saved_data" + "\\" + filename
                cursor = cnxn.cursor()
    
                SQL_string= ("BULK INSERT [Stock_Data].[dbo].[Hist_Data] FROM '""" + 
                directory + "\\" + filename + """' WITH 
                (FIRSTROW=2,FIELDTERMINATOR = ',',ROWTERMINATOR = '\\n')""")
                
                cursor.execute(SQL_string)
                cnxn.commit()
                cursor.close()
            
            except Exception:
                print(filename[:-4])
                cursor.close()
            continue
        else:
            continue
    cnxn.close()

In [17]:
#times out instead of throwing error.  Will examine more later if needed.  
df_morningstar=pdr.DataReader('AAPL','morningstar',start_date,end_date)

In [12]:
#annoying ticker requirement.  Willl examine more later if needed.  
param = {
    'q': "TURN", # Stock symbol (ex: "AAPL")
    'i': "86400", # Interval size in seconds ("86400" = 1 day intervals)
    'x': "NASD", # Stock exchange symbol on which stock is traded (ex: "NASD")
    'p': "1Y" # Period (Ex: "1Y" = 1 year)
}
# get price data (return pandas dataframe)
df_google = get_price_data(param)


